In [1]:
import pandas as pd

In [40]:
stages_profile = pd.read_csv('data/parcours_clean.csv').groupby('year')['Parcours type:', 'ProfileScore:', 'Vert. meters:'].mean().reset_index()
stages_profile['year'] = stages_profile['year'].astype('str')
profile = pd.read_csv('data/data_profile.csv').drop_duplicates()
age = pd.read_csv('data/data_age.csv').drop_duplicates()
#performance = pd.read_csv('data_performance.csv').drop_duplicates()
teams = pd.read_csv('data/data_teams.csv').drop_duplicates()
team_strength = pd.read_csv('data/data_team_strength.csv', index_col=0).reset_index()
finishes = pd.read_csv('data/data_finishes.csv', index_col=0).reset_index()
master_ls = []

for year in [2017,2018,2019,2020,2021]:
    points_last = pd.read_csv(f'data/data_points_{year-1}.csv', index_col=0).drop(columns='year')
    points_current = pd.read_csv(f'data/data_points_{year}.csv', index_col=0).drop(columns='year').rename(columns={'scorito': 'target'})
    table2 = pd.merge(profile, age,  how='outer', left_on=['name','year', 'team'], right_on = ['name','year', 'team'])
    table4 = pd.merge(table2, teams, how='outer', left_on=['team','year'], right_on = ['team','year']).drop(columns=['ranking', 'team_class'])
    table4['year'] = table4['year'].astype('str')
    table4['key'] = table4[['name', 'year']].agg('-'.join, axis=1)
    table5 = table4.pivot_table(values='points', index='key', columns='type', aggfunc='first').reset_index()
    table6 = pd.merge(table4, table5, how='outer', left_on='key', right_on = 'key').drop(columns=['type', 'points']).drop_duplicates()
    table7 = table6.merge(stages_profile, how='left', on='year').rename(columns={'Parcours type:':'profile_type', 'ProfileScore:' : 'profile_score', 'Vert. meters:':'profile_vert', 'name':'rider'})
    table8 = table7.merge(points_last, how='left', on='rider')
    table9 = table8.merge(points_current, how='left', on='rider')
    team_strength['year'] = team_strength['year'].astype('str')
    table10 = table9.merge(team_strength, how='left', on=['team', 'year'])
    finishes['year'] = finishes['year'].astype('str')
    table11 = table10.merge(finishes, how='outer', on=['rider', 'year']).sort_values('finished', ascending=False)
    
    master_ls.append(table11)

master_df = pd.concat(master_ls)
master_df = master_df[master_df['target'].notna()]
master_df.to_csv('data/training.csv')

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_83379/657948037.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stages_profile = pd.read_csv('data/parcours_clean.csv').groupby('year')['Parcours type:', 'ProfileScore:', 'Vert. meters:'].mean().reset_index()


In [39]:
table10.merge(finishes, how='outer', on=['rider', 'year']).sort_values('finished', ascending=False)

,rider,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders,profile_type,profile_score,profile_vert,scorito,target,strength,finished
666,Dupont Hubert,AG2R La Mondiale,2019,38,1486,Dupont Hubert-2019,14.0,NaN,NaN,NaN,2.952381,138.190476,2478.047619,NaN,NaN,36.0,0.916667
530,Dupont Hubert,AG2R La Mondiale,2018,37,1459,Dupont Hubert-2018,NaN,NaN,NaN,NaN,3.142857,135.476190,2437.476190,NaN,NaN,28.0,0.909091
213,Pellizotti Franco,Bahrain Merida Pro Cycling Team,2017,39,2364,Pellizotti Franco-2017,83.0,NaN,NaN,54.0,2.952381,134.285714,2570.619048,NaN,NaN,53.0,0.818182
285,Tiralongo Paolo,Astana Pro Team,2017,39,1765,Tiralongo Paolo-2017,10.0,NaN,NaN,NaN,2.952381,134.285714,2570.619048,NaN,NaN,33.0,0.818182
291,Dupont Hubert,AG2R La Mondiale,2017,36,1603,Dupont Hubert-2017,NaN,NaN,38.0,NaN,2.952381,134.285714,2570.619048,NaN,NaN,32.0,0.818182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,Christian Mark,EOLO-Kometa,2021,30,124,Christian Mark-2021,NaN,NaN,NaN,NaN,2.809524,153.809524,2261.761905,NaN,22.0,66.0,NaN
1095,Gabburo Davide,Bardiani-CSF-Faizanè,2021,28,644,Gabburo Davide-2021,69.0,8.0,51.0,5.0,2.809524,153.809524,2261.761905,NaN,0.0,89.0,NaN
1097,Marengo Umberto,Bardiani-CSF-Faizanè,2021,28,644,Marengo Umberto-2021,15.0,NaN,25.0,NaN,2.809524,153.809524,2261.761905,NaN,4.0,89.0,NaN
1098,Zoccarato Samuele,Bardiani-CSF-Faizanè,2021,23,644,Zoccarato Samuele-2021,NaN,NaN,NaN,NaN,2.809524,153.809524,2261.761905,NaN,2.0,89.0,NaN


In [37]:
finishes = pd.read_csv('data/data_finishes.csv', index_col=0).reset_index()
finishes['year'] = finishes['year'].astype('str')
finishes

,rider,finished,year
0,Tiralongo Paolo,0.818182,2017
1,Pellizotti Franco,0.818182,2017
2,Pozzovivo Domenico,0.545455,2017
3,Dupont Hubert,0.818182,2017
4,Bak Lars Ytting,0.545455,2017
...,...,...,...
634,Guglielmi Simon,0.071429,2021
635,Oldani Stefano,0.071429,2021
636,Foss Tobias,0.000000,2021
637,Zana Filippo,0.071429,2021


In [92]:
master_df[master_df['target'].notna()]

,rider,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders,profile_type,profile_score,profile_vert,scorito,target
1,Amador Andrey,Movistar Team,2016,29,5145,Amador Andrey-2016,36.0,255.0,NaN,98.0,2.857143,137.095238,2244.285714,326.0,160.0
2,Moreno Javier,Movistar Team,2016,31,5145,Moreno Javier-2016,13.0,108.0,NaN,74.0,2.857143,137.095238,2244.285714,18.0,10.0
4,Rojas José Joaquín,Movistar Team,2016,30,5145,Rojas José Joaquín-2016,206.0,53.0,389.8,NaN,2.857143,137.095238,2244.285714,36.0,44.0
5,Herrada José,Movistar Team,2016,30,5145,Herrada José-2016,100.0,42.0,NaN,NaN,2.857143,137.095238,2244.285714,18.0,46.0
6,Visconti Giovanni,Movistar Team,2016,33,5145,Visconti Giovanni-2016,175.0,NaN,NaN,NaN,2.857143,137.095238,2244.285714,186.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,Niv Guy,Israel Start-Up Nation,2021,27,1405,Niv Guy-2021,NaN,30.0,NaN,25.0,2.809524,153.809524,2261.761905,NaN,44.0
1103,De Marchi Alessandro,Israel Start-Up Nation,2021,34,1405,De Marchi Alessandro-2021,110.0,25.0,NaN,47.0,2.809524,153.809524,2261.761905,NaN,98.0
1104,Cimolai Davide,Israel Start-Up Nation,2021,31,1405,Cimolai Davide-2021,19.0,20.0,146.0,NaN,2.809524,153.809524,2261.761905,73.0,363.0
1105,Bevin Patrick,Israel Start-Up Nation,2021,30,1405,Bevin Patrick-2021,NaN,9.0,8.0,85.0,2.809524,153.809524,2261.761905,NaN,72.0


In [60]:
stages_profile = pd.read_csv('parcours_clean.csv').groupby('year')['Parcours type:', 'ProfileScore:', 'Vert. meters:'].mean().reset_index()
stages_profile['year'] = stages_profile['year'].astype('str')
stages_profile.dtypes

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_67129/4216735733.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stages_profile = pd.read_csv('parcours_clean.csv').groupby('year')['Parcours type:', 'ProfileScore:', 'Vert. meters:'].mean().reset_index()


year               object
Parcours type:    float64
ProfileScore:     float64
Vert. meters:     float64
dtype: object

In [81]:
points_last = pd.read_csv('data_points_2016.csv', index_col=0).drop(columns='year')
points_current = pd.read_csv('data_points_2017.csv', index_col=0).drop(columns='year').rename(columns={'scorito': 'target'})

In [91]:
points_current

,rider,target
39,Bernal Egan,1188.0
32,Martínez Daniel Felipe,562.0
24,Caruso Damiano,519.0
50,Sagan Peter,497.0
10,Vlasov Aleksandr,495.0
...,...,...
91,Serry Pieter,0.0
135,Ghebreigzabhier Amanuel,0.0
85,Martens Paul,0.0
137,Villella Davide,0.0


In [44]:
profile = pd.read_csv('data_profile.csv').drop_duplicates()
age = pd.read_csv('data_age.csv').drop_duplicates()
#performance = pd.read_csv('data_performance.csv').drop_duplicates()
teams = pd.read_csv('data_teams.csv').drop_duplicates()

In [45]:
profile = profile[profile['year']==2017]
age = age[age['year']==2017]
teams = teams[teams['year']==2017]

In [46]:
profile

,name,team,points,ranking,type,year
416,Quintana Nairo,Movistar Team,1550.0,1,top-gc-riders,2017
417,Nibali Vincenzo,Bahrain Merida Pro Cycling Team,803.0,7,top-gc-riders,2017
418,Pinot Thibaut,FDJ,722.0,8,top-gc-riders,2017
419,Thomas Geraint,Team Sky,547.0,12,top-gc-riders,2017
420,Costa Rui,UAE Team Emirates,499.0,15,top-gc-riders,2017
...,...,...,...,...,...,...
841,Tiralongo Paolo,Astana Pro Team,10.0,967,best-classic-riders,2017
842,Senni Manuel,BMC Racing Team,10.0,955,best-classic-riders,2017
843,Verona Carlos,ORICA-Scott,10.0,972,best-classic-riders,2017
844,Chevrier Clément,AG2R La Mondiale,10.0,957,best-classic-riders,2017


In [47]:
table2 = pd.merge(profile, age,  how='outer', left_on=['name','year', 'team'], right_on = ['name','year', 'team'])
table2.head()

,name,team,points,ranking,type,year,age
0,Quintana Nairo,Movistar Team,1550.0,1.0,top-gc-riders,2017,27
1,Quintana Nairo,Movistar Team,34.6,907.0,top-sprinters,2017,27
2,Quintana Nairo,Movistar Team,358.0,20.0,top-time-trial-riders,2017,27
3,Nibali Vincenzo,Bahrain Merida Pro Cycling Team,803.0,7.0,top-gc-riders,2017,32
4,Nibali Vincenzo,Bahrain Merida Pro Cycling Team,44.8,739.0,top-sprinters,2017,32


In [48]:
table4 = pd.merge(table2, teams, how='outer', left_on=['team','year'], right_on = ['team','year']).drop(columns=['ranking', 'team_class'])
table4.head()

,name,team,points,type,year,age,team_points
0,Quintana Nairo,Movistar Team,1550.0,top-gc-riders,2017,27,3999
1,Quintana Nairo,Movistar Team,34.6,top-sprinters,2017,27,3999
2,Quintana Nairo,Movistar Team,358.0,top-time-trial-riders,2017,27,3999
3,Izagirre Gorka,Movistar Team,267.0,top-gc-riders,2017,29,3999
4,Izagirre Gorka,Movistar Team,39.3,top-sprinters,2017,29,3999


In [49]:
table4['year'] = table4['year'].astype('str')
table4['key'] = table4[['name', 'year']].agg('-'.join, axis=1)
table4

,name,team,points,type,year,age,team_points,key
0,Quintana Nairo,Movistar Team,1550.0,top-gc-riders,2017,27,3999,Quintana Nairo-2017
1,Quintana Nairo,Movistar Team,34.6,top-sprinters,2017,27,3999,Quintana Nairo-2017
2,Quintana Nairo,Movistar Team,358.0,top-time-trial-riders,2017,27,3999,Quintana Nairo-2017
3,Izagirre Gorka,Movistar Team,267.0,top-gc-riders,2017,29,3999,Izagirre Gorka-2017
4,Izagirre Gorka,Movistar Team,39.3,top-sprinters,2017,29,3999,Izagirre Gorka-2017
...,...,...,...,...,...,...,...,...
440,Zhupa Eugert,Wilier Triestina - Selle Italia,60.0,top-time-trial-riders,2017,27,1391,Zhupa Eugert-2017
441,Zhupa Eugert,Wilier Triestina - Selle Italia,33.0,best-classic-riders,2017,27,1391,Zhupa Eugert-2017
442,Fonzi Giuseppe,Wilier Triestina - Selle Italia,36.0,top-sprinters,2017,25,1391,Fonzi Giuseppe-2017
443,Fonzi Giuseppe,Wilier Triestina - Selle Italia,26.0,top-time-trial-riders,2017,25,1391,Fonzi Giuseppe-2017


In [50]:
table5 = table4.pivot_table(values='points', index='key', columns='type', aggfunc='first').reset_index()
table5

type,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders
0,Albanese Vincenzo-2017,90.0,NaN,246.0,30.0
1,Amador Andrey-2017,37.0,180.0,85.0,202.0
2,Amezqueta Julen-2017,13.0,48.0,NaN,NaN
3,Anacona Winner-2017,16.0,123.0,NaN,NaN
4,Andreetta Simone-2017,35.0,NaN,NaN,NaN
...,...,...,...,...,...
177,Zeits Andrey-2017,100.0,100.0,NaN,10.0
178,Zhupa Eugert-2017,33.0,NaN,46.0,60.0
179,de la Parte Víctor-2017,29.0,145.0,NaN,24.0
180,van Emden Jos-2017,NaN,216.0,NaN,419.0


In [63]:
table6 = pd.merge(table4, table5, how='outer', left_on='key', right_on = 'key').drop(columns=['type', 'points']).drop_duplicates()
table6

,name,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders
0,Quintana Nairo,Movistar Team,2017,27,3999,Quintana Nairo-2017,NaN,1550.0,34.6,358.0
3,Izagirre Gorka,Movistar Team,2017,29,3999,Izagirre Gorka-2017,76.0,267.0,39.3,172.0
7,Amador Andrey,Movistar Team,2017,30,3999,Amador Andrey-2017,37.0,180.0,85.0,202.0
11,de la Parte Víctor,Movistar Team,2017,30,3999,de la Parte Víctor-2017,29.0,145.0,NaN,24.0
14,Anacona Winner,Movistar Team,2017,28,3999,Anacona Winner-2017,16.0,123.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
435,Pozzato Filippo,Wilier Triestina - Selle Italia,2017,35,1391,Pozzato Filippo-2017,299.0,NaN,147.0,NaN
437,Busato Matteo,Wilier Triestina - Selle Italia,2017,29,1391,Busato Matteo-2017,155.0,NaN,90.7,NaN
439,Zhupa Eugert,Wilier Triestina - Selle Italia,2017,27,1391,Zhupa Eugert-2017,33.0,NaN,46.0,60.0
442,Fonzi Giuseppe,Wilier Triestina - Selle Italia,2017,25,1391,Fonzi Giuseppe-2017,NaN,NaN,36.0,26.0


In [70]:
table7 = table6.merge(stages_profile, how='left', on='year').rename(columns={'Parcours type:':'profile_type', 'ProfileScore:' : 'profile_score', 'Vert. meters:':'profile_vert', 'name':'rider'})
table7

,rider,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders,profile_type,profile_score,profile_vert
0,Quintana Nairo,Movistar Team,2017,27,3999,Quintana Nairo-2017,NaN,1550.0,34.6,358.0,2.952381,134.285714,2570.619048
1,Izagirre Gorka,Movistar Team,2017,29,3999,Izagirre Gorka-2017,76.0,267.0,39.3,172.0,2.952381,134.285714,2570.619048
2,Amador Andrey,Movistar Team,2017,30,3999,Amador Andrey-2017,37.0,180.0,85.0,202.0,2.952381,134.285714,2570.619048
3,de la Parte Víctor,Movistar Team,2017,30,3999,de la Parte Víctor-2017,29.0,145.0,NaN,24.0,2.952381,134.285714,2570.619048
4,Anacona Winner,Movistar Team,2017,28,3999,Anacona Winner-2017,16.0,123.0,NaN,NaN,2.952381,134.285714,2570.619048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Pozzato Filippo,Wilier Triestina - Selle Italia,2017,35,1391,Pozzato Filippo-2017,299.0,NaN,147.0,NaN,2.952381,134.285714,2570.619048
193,Busato Matteo,Wilier Triestina - Selle Italia,2017,29,1391,Busato Matteo-2017,155.0,NaN,90.7,NaN,2.952381,134.285714,2570.619048
194,Zhupa Eugert,Wilier Triestina - Selle Italia,2017,27,1391,Zhupa Eugert-2017,33.0,NaN,46.0,60.0,2.952381,134.285714,2570.619048
195,Fonzi Giuseppe,Wilier Triestina - Selle Italia,2017,25,1391,Fonzi Giuseppe-2017,NaN,NaN,36.0,26.0,2.952381,134.285714,2570.619048


In [85]:
table8 = table7.merge(points_last, how='left', on='rider')
table9 = table8.merge(points_current, how='left', on='rider')

In [86]:
table9

,rider,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders,profile_type,profile_score,profile_vert,scorito,target
0,Quintana Nairo,Movistar Team,2017,27,3999,Quintana Nairo-2017,NaN,1550.0,34.6,358.0,2.952381,134.285714,2570.619048,NaN,584.0
1,Izagirre Gorka,Movistar Team,2017,29,3999,Izagirre Gorka-2017,76.0,267.0,39.3,172.0,2.952381,134.285714,2570.619048,NaN,138.0
2,Amador Andrey,Movistar Team,2017,30,3999,Amador Andrey-2017,37.0,180.0,85.0,202.0,2.952381,134.285714,2570.619048,326.0,160.0
3,de la Parte Víctor,Movistar Team,2017,30,3999,de la Parte Víctor-2017,29.0,145.0,NaN,24.0,2.952381,134.285714,2570.619048,NaN,NaN
4,Anacona Winner,Movistar Team,2017,28,3999,Anacona Winner-2017,16.0,123.0,NaN,NaN,2.952381,134.285714,2570.619048,NaN,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Pozzato Filippo,Wilier Triestina - Selle Italia,2017,35,1391,Pozzato Filippo-2017,299.0,NaN,147.0,NaN,2.952381,134.285714,2570.619048,0.0,0.0
193,Busato Matteo,Wilier Triestina - Selle Italia,2017,29,1391,Busato Matteo-2017,155.0,NaN,90.7,NaN,2.952381,134.285714,2570.619048,74.0,58.0
194,Zhupa Eugert,Wilier Triestina - Selle Italia,2017,27,1391,Zhupa Eugert-2017,33.0,NaN,46.0,60.0,2.952381,134.285714,2570.619048,0.0,15.0
195,Fonzi Giuseppe,Wilier Triestina - Selle Italia,2017,25,1391,Fonzi Giuseppe-2017,NaN,NaN,36.0,26.0,2.952381,134.285714,2570.619048,NaN,0.0
